**K-State Honor Code "On my honor, as a student, I have neither given nor received unauthorized aid on this ascademic work"**

# 1. Business Questions about Nike

a. First Question: Who tweets the most about Nike?
    
b. Second Question: Does Nike have more positive, negative, or neutural tweets?
    

# 2. Important Dataset

In [82]:
#Libraries
import tweepy
import pandas as pd
import json
import regex
import re

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yutowada/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [83]:
# put your credential API information here (do not share your API to the public)
# API Consumer Key and Secret from Twitter
api_key = "WsEUv6afCiNthkSLWHIHn8yUM"
api_secret = "jtg2kd1CFL1txDao6NouIUO7xZzCXSM4yiP4WyuiiqQ50qTtjf"

access_token = "1564294883636088838-jhUHzc9xuIH9WMu6wCPahsWM4LiwaG"
access_token_secret = "vM6EmVcxk3L94D1bVQz19MliIUcSW5uA9660IZk1FQken"

# authentication
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth,wait_on_rate_limit=True)

In [84]:
tweets_data = [] #initialize an empty list

for status in tweepy.Cursor(api.search_tweets,q="Nike",
                            until='2022-11-03', #Twitter will automatically sample the last 7 days of data, and only allows you to get 7-day data
                            result_type='recent',
                            include_entities=True,
                            tweet_mode='extended', #otherwise it only captures 140 characters
                            lang="en").items(600):
    
    post_time = status.created_at # tweets posting time
    tweet_id = status.id_str # gets the tweets thread ID
    tweet_text= status.full_text # gets the tweets texts
    retweet_count = status.retweet_count
    user_name = status.user.name
    user_screen_name = status.user.screen_name
    user_location = status.user.location
    user_followers_count = status.user.followers_count
    
    tweets_data.append((post_time, tweet_id, tweet_text, retweet_count, user_name, user_screen_name, user_location, user_followers_count))

headers = ['Post Time', 'Tweet ID', 'TweetText', 'RetweetCount', 'UserName', 'UserScreenName', 'User Location', 'UserFollowerCounts']

alltweets = pd.DataFrame(tweets_data,columns = headers)
alltweets.head()

,Post Time,Tweet ID,TweetText,RetweetCount,UserName,UserScreenName,User Location,UserFollowerCounts
0,2022-11-02 23:59:59+00:00,1587957725664890881,RT @ArndTownSnks: @CurrenSy_Spitta &amp; @ther...,204,negin,neginsoleimani,texas,4492
1,2022-11-02 23:59:56+00:00,1587957715136978946,all this no L Cancelling shit is so annoying j...,0,shinji,aeShinjii,"Tokyo-to, Japan",66
2,2022-11-02 23:59:53+00:00,1587957700360536066,@ImarLearning @Nike Good answer...That touches...,2,Budget The Bag™,BudgetTheBag,Texas,81
3,2022-11-02 23:59:40+00:00,1587957648887930880,@DrifterShoots @Nike Bro got the huaraches on....,0,🆂🅱🅾🅹 ,4GIVEMYSOUL,The Universe,483
4,2022-11-02 23:59:39+00:00,1587957644731498496,🔹 Men's Nike Air Max LTD 3 Casual Shoes\n🔹 Men...,0,Free Nike Monitor,free_nike_bot,,264


# 3.Text Processing

In [85]:
#Remove stopwords
stop_words = stopwords.words('english')
more_stopwords = set(('cant', 'aint', 'today', 'one', 'like', 'could', 'see', 'also', 
'get', 'yu', 'u', 'st', 'lot'))
extra_stoplist = set(stopwords.words('english')) | more_stopwords
alltweets['TweetText'] = alltweets['TweetText'].apply(lambda x: ' '.join([word for word in x.split() if word not in (extra_stoplist)]))

In [86]:
#Remove symbols and numbers
alltweets["TweetText"] = alltweets["TweetText"].str.replace(r'https?://[^\s<>"]+|www\.[^\s<>"]+', "")
alltweets["TweetText"] = alltweets["TweetText"].str.replace(r'[^a-zA-Z0-9]+', ' ')

/var/folders/1j/79l9x7_16cldqkzdldfx37vh0000gn/T/ipykernel_54296/1895569368.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  alltweets["TweetText"] = alltweets["TweetText"].str.replace(r'https?://[^\s<>"]+|www\.[^\s<>"]+', "")
/var/folders/1j/79l9x7_16cldqkzdldfx37vh0000gn/T/ipykernel_54296/1895569368.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  alltweets["TweetText"] = alltweets["TweetText"].str.replace(r'[^a-zA-Z0-9]+', ' ')


# 4.Descriptive Analytics

In [87]:
#How many users in the dataset
print(len(alltweets.UserScreenName.unique()))

509


In [88]:
#Who is the most popular user in the dataset?
alltweets.loc[alltweets["UserFollowerCounts"].idxmax()].UserScreenName

'HYPEBEAST'

In [89]:
#Tweets per users
Number_tweets= len(alltweets['TweetText'].unique())
Number_users = len(alltweets['UserScreenName'].unique())

round(float(Number_tweets/Number_users),3)

0.809

In [90]:
#Number of Original Tweets
# view tweets do not contain RT (original tweets)
new_dftweets = alltweets[~alltweets["TweetText"].str.contains("RT")]
print(len(new_dftweets))

284


In [91]:
#Retweet count for every tweet
alltweets = alltweets.sort_values(by = 'RetweetCount', ascending=False)
alltweets.head()

,Post Time,Tweet ID,TweetText,RetweetCount,UserName,UserScreenName,User Location,UserFollowerCounts
152,2022-11-02 23:48:45+00:00,1587954901224677378,RT ny ocho Nike Dunk Low The Mystery Machine,12265,Koko🌙 Main Kokomi,kokobabb,,1725
71,2022-11-02 23:54:07+00:00,1587956248246456321,RT ny ocho Nike Dunk Low The Mystery Machine,12265,Harriet 🌙,harriet_vasey,"Newcastle Upon Tyne, England",102
67,2022-11-02 23:54:42+00:00,1587956395860557824,RT KTLA Update A second suspect identified dou...,3396,Migayla,Nerd_Nike,Looking for Wonderland,733
332,2022-11-02 23:34:38+00:00,1587951348846399489,RT ModestMama Calling Pick Me sale 5 99 month ...,2981,Daario Naharis,Tweets_n_Nike,"New York, USA",1590
394,2022-11-02 23:30:45+00:00,1587950371846045698,RT TalbertSwan White privilege Brett Favre can...,2930,Channy B,ChantalBorgella,Jersey Girl ✈ Los Angeles,1756


# 5.Sentiment Analysis

In [92]:

alltweets['TweetText'] = alltweets['TweetText'].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))
alltweets["TweetText"] = alltweets["TweetText"].str.replace(r'https?://[^\s<>"]+|www\.[^\s<>"]+', "")
alltweets['TweetText'] = alltweets["TweetText"].str.replace(r'[^a-zA-Z0-9]+', ' ')
alltweets['TweetText'] = alltweets["TweetText"].str.replace('@#', '')
alltweets['TweetText'] = alltweets["TweetText"].str.replace('\n', '')
print(len(alltweets))

600


/var/folders/1j/79l9x7_16cldqkzdldfx37vh0000gn/T/ipykernel_54296/3081627793.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  alltweets["TweetText"] = alltweets["TweetText"].str.replace(r'https?://[^\s<>"]+|www\.[^\s<>"]+', "")
/var/folders/1j/79l9x7_16cldqkzdldfx37vh0000gn/T/ipykernel_54296/3081627793.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  alltweets['TweetText'] = alltweets["TweetText"].str.replace(r'[^a-zA-Z0-9]+', ' ')


In [93]:
positive_words = open('Bing_liu_positive.txt', "r").read().splitlines()
negative_words = open('Bing_liu_negative.txt', "r").read().splitlines()
list_tweet = alltweets['TweetText'].to_list()

new_texts = []
remove = ['br', 's']
flat_review = [''.join(ele) for ele in list_tweet]
documents = [re.sub("[^a-zA-Z]+", " ", document) for document in flat_review]

texts = [[word for word in document.split() ] for document in documents]

print(len(list_tweet))

600


In [94]:
positive_review = []
negative_review = []
neutral_review = []

for sentence in texts:
    positive = 0
    negative = 0
    for word in sentence:
        if word in positive_words:
            positive += 1
        elif word in negative_words:
            negative += 1
    if positive-negative > 0:
            positive_review.append(sentence)
    elif positive-negative == 0:
        neutral_review.append(sentence)
    else:
        negative_review.append(sentence)

## Number of positive, negative, and neutural tweets

In [95]:
print(len(positive_review), ' positive tweet')
print(len(negative_review), ' negative tweet')
print(len(neutral_review), ' neutural tweet')

158  positive tweet
55  negative tweet
387  neutural tweet


In [96]:
#Joining back the split review into one sentence
# and appending them into new review
positive_review_complete = []
negative_review_complete = []
neutral_review_complete = []

for reviews in positive_review:
    join_string = " "
    complete_review = join_string.join(reviews)
    positive_review_complete.append(complete_review)

for reviews in negative_review:
    join_string = " "
    complete_review = join_string.join(reviews)
    negative_review_complete.append(complete_review)

for reviews in neutral_review:
    join_string = " "
    complete_review = join_string.join(reviews)
    neutral_review_complete.append(complete_review)

In [97]:
alltweets["Analysis"] = ""
alltweets.head()

,Post Time,Tweet ID,TweetText,RetweetCount,UserName,UserScreenName,User Location,UserFollowerCounts,Analysis
152,2022-11-02 23:48:45+00:00,1587954901224677378,RT ny ocho Nike Dunk Low The Mystery Machine,12265,Koko🌙 Main Kokomi,kokobabb,,1725,
71,2022-11-02 23:54:07+00:00,1587956248246456321,RT ny ocho Nike Dunk Low The Mystery Machine,12265,Harriet 🌙,harriet_vasey,"Newcastle Upon Tyne, England",102,
67,2022-11-02 23:54:42+00:00,1587956395860557824,RT KTLA Update A second suspect identified dou...,3396,Migayla,Nerd_Nike,Looking for Wonderland,733,
332,2022-11-02 23:34:38+00:00,1587951348846399489,RT ModestMama Calling Pick Me sale 5 99 month ...,2981,Daario Naharis,Tweets_n_Nike,"New York, USA",1590,
394,2022-11-02 23:30:45+00:00,1587950371846045698,RT TalbertSwan White privilege Brett Favre can...,2930,Channy B,ChantalBorgella,Jersey Girl ✈ Los Angeles,1756,


In [98]:
#Document which tweet is positive, negative or neutural
documents = [re.sub("[^a-zA-Z]+", " ", document) for document in flat_review]
new_documents = [review.strip() for review in documents]

for index, review in enumerate(new_documents):
    found = False
    for positive in positive_review_complete:
        if positive in review:
            found = True
            alltweets.loc[index, 'Analysis'] = "Positive"
        else:
            continue
    if not found:
        for negative in negative_review_complete:
            if negative in review:
                found = True
                alltweets.loc[index, 'Analysis'] = "Negative"
            else:
                continue
    if not found:
        for neutural in neutral_review_complete:
            if neutural in review:
                found = True
                alltweets.loc[index, 'Analysis'] = "Neutural"
            else:
                continue
alltweets.head()

,Post Time,Tweet ID,TweetText,RetweetCount,UserName,UserScreenName,User Location,UserFollowerCounts,Analysis
152,2022-11-02 23:48:45+00:00,1587954901224677378,RT ny ocho Nike Dunk Low The Mystery Machine,12265,Koko🌙 Main Kokomi,kokobabb,,1725,Neutural
71,2022-11-02 23:54:07+00:00,1587956248246456321,RT ny ocho Nike Dunk Low The Mystery Machine,12265,Harriet 🌙,harriet_vasey,"Newcastle Upon Tyne, England",102,Positive
67,2022-11-02 23:54:42+00:00,1587956395860557824,RT KTLA Update A second suspect identified dou...,3396,Migayla,Nerd_Nike,Looking for Wonderland,733,Positive
332,2022-11-02 23:34:38+00:00,1587951348846399489,RT ModestMama Calling Pick Me sale 5 99 month ...,2981,Daario Naharis,Tweets_n_Nike,"New York, USA",1590,Neutural
394,2022-11-02 23:30:45+00:00,1587950371846045698,RT TalbertSwan White privilege Brett Favre can...,2930,Channy B,ChantalBorgella,Jersey Girl ✈ Los Angeles,1756,Neutural


# 6.Storytelling

From the result data collection, text processing, descriptive analysis, and sentiment analysis, I was able to answer two question that I wanted to answer.

a. First Question: Who tweets the most about Nike?

b. Second Question: Does Nike have more positive, negative, or neutural tweets?

By performing the desctiptive analysis, I was able to answer the first question. User with name **'HYPEBEAST'** was the user that was tweeting the most about Nike. 

As for the second question 2, I used Sentiment Analysis to answer this one. Result came out suprising, as over half of the tweets were considered to be neutrual. Further looking into the tweets, it looks to be that most of tweets where tweets about sneaker release details.

1. 158  positive tweet

2. 55  negative tweet

3. 387  neutural tweet

Because most of the tweets were information about sneaker release, there were many tweets with more then 1000 retweets. This indicates that many people are using that original tweet to spread information about upcoming sneaker release.
